In [ ]:
def roomparse(room):
    roomdict = {}
    try:
        detli = room.find('div',{'class':'room-details'}).find_all('li')
    except:
        detli = []
        roomdict['details_missing'] = 1
    for li in detli:
        roomdict[li.text] = 1

    try:
        inputli = room.find('form').find_all('input')
    except:
        inputli = []
    for inp in inputli:
        try:
            roomdict[inp['name']] = inp['value']
        except:
            pass

    for feature in room.find_all('span',{'class':'feature-title'}):
        roomdict[feature.text] = 1

    for fun,name in [[lambda x: x.find('h3').text,'room_name'],
                    [lambda x: x.find('span',{'class':'occupancy-rate'}).text,'occupancy'],
                    [lambda x: x.find('div',{'class':'cancellation'}).text.split()[0],'cancellation'],
                    [lambda x: x.find('strong',{'class':'current-price'}).text,'price'],
                    [lambda x: x.find('div',{'class':'price'}).text,'price-2'],
                    [lambda x: x['data-index'],'data-index']]:
        try:
            roomdict[name] = fun(room)
        except:
            pass
    return roomdict

def process_date(inlists):
    out = []
    for inlist in tqdm(inlists):
        hotelid = inlist[0]
        date=inlist[1]
        chin = date.strftime('%Y-%m-%d')
        chout = (date + datetime.timedelta(days = 1)).strftime('%Y-%m-%d')
        params = {'q-check-out':chout,
                  'q-check-in':chin,
                  'q-room-0-adults':2,
                  'q-rooms':1,
                  'q-room-0-children':0,
                  'locale':'en_IE'}
        url = 'https://www.hotels.com/ho' + str(hotelid)

        r = requests.get(url,params=params)
        #print(r.url)
        rooms = BeautifulSoup(r.content,'html5lib').find_all('li',{'class':'room'})
        hotelinfo = {'hotelid':hotelid,
                    'date':chin,
                    'rooms':len(rooms)}
        out.append([pd.DataFrame([roomparse(r) for r in rooms]),hotelinfo])
    return out

def process_dateoutput(outlists):
    return [pd.concat([x[0] for outlist in outlists for x in outlist]),
            pd.DataFrame([x[1] for outlist in outlists for x in outlist])]

In [ ]:
hotelsamplesize = HOTEL_DATE_SAMPLE
daterange = DATE_RANGE

pairs = itertools.product(hotel_atts.sample(hotelsamplesize).index,
                  [(datetime.date.today() + datetime.timedelta(days = d))
                   for d in daterange])

pairs_split = np.array_split(list(pairs),SPLIT_NUM)

pool2 = multiprocessing.Pool(POOL_NUM)

roomdf,hoteldates = process_dateoutput(pool2.map(process_date,[l for l in pairs_split]))